In [2]:
import csv
import math
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [3]:
datos=[]
with open('2022 MLB Player Stats - Batting.csv',newline='',encoding='utf-8') as archivo_csv:
    lector_csv=csv.DictReader(archivo_csv,delimiter=';')
    for fila in lector_csv:
        datos.append(fila)


#casteando la entrada
llaves=list(datos[0])
for columna in llaves:
    if not columna in ["Name", "Tm", "Lg"]:
        if not columna in ["BA", "OBP", "SLG", "OPS", "OPS+"]:
            for fila in datos:
                fila[columna]=int(fila[columna])
        else:
            for fila in datos:
                fila[columna]=float(fila[columna])

In [4]:
Over25_2D_NL=list(filter(lambda x: x['2B']>25 and x["Lg"]=="NL" , datos))

#El primer supuesto menciona que debe existir una relacion lineal entre las variables 
#dependientes y las independientes. Y solo hay dos variables que cumplen, por lo que al 
#aplicar el metodo foward no importa el orden en que tome la variable
#Las variables tomadas seran RBI y TB

Matriz=np.array(
    [
        [
            len(Over25_2D_NL),
            sum(map(lambda x:x['RBI'],Over25_2D_NL)),
            sum(map(lambda x:x['TB'],Over25_2D_NL))
        ],
        [
            sum(map(lambda x:x['RBI'],Over25_2D_NL)),
            sum(map(lambda x:x['RBI']*x['RBI'],Over25_2D_NL)),
            sum(map(lambda x:x['RBI']*x['TB'],Over25_2D_NL))
        ],
        [
            sum(map(lambda x:x['TB'],Over25_2D_NL)),
            sum(map(lambda x:x['TB']*x['RBI'],Over25_2D_NL)),
            sum(map(lambda x:x['TB']*x['TB'],Over25_2D_NL))
        ]
    ]
)

vector=np.array(
    [
        sum(map(lambda x:x['HR'],Over25_2D_NL)),
        sum(map(lambda x:x['RBI']*x['HR'],Over25_2D_NL)),
        sum(map(lambda x:x['TB']*x['HR'],Over25_2D_NL))
    ]   
)

c=np.linalg.solve(Matriz,vector)

print('a)')
print("La regresion lineal multiple de RBI y TB resulto:")
print("HR = "+str(c[0])+" + "+str(c[1])+"*RBI + "+str(c[2])+"*TB")

a)
La regresion lineal multiple de RBI y TB resulto:
HR = -14.093237249753416 + 0.2542475451793053*RBI + 0.06350788243921907*TB


In [8]:
#Aprovechemos la matriz y el vector del inciso a

Matriz[0][2]=Matriz[2][0]=sum(map(lambda x:x['SO'],Over25_2D_NL))
Matriz[1][2]=Matriz[2][1]=Matriz[0][1]*Matriz[0][2]
Matriz[2][2]=Matriz[2][0]=sum(map(lambda x:x['SO']*x['SO'],Over25_2D_NL))
vector[2]=sum(map(lambda x:x['SO']*x['HR'],Over25_2D_NL))
c2=np.linalg.solve(Matriz,vector)
print()
print("La regresion lineal multiple realizada con RBI y SO resulto:")
print("HR = "+str(c2[0])+" + "+str(c2[1])+"*RBI + "+str(c2[2])+"*SO")

ECM=sum(map(lambda x:x*x,list(map(lambda x:x['HR']-c[0]-c[1]*x['RBI']-c[2]*x['TB'],Over25_2D_NL))))
ECM2=sum(list(map(lambda x:pow(x['HR']-c2[0]-c2[1]*x['RBI']-c2[2]*x['SO'],2),Over25_2D_NL)))

print("El error cuadratico medio de RBI y TB es:",ECM)
print("El error cuadratico medio de RBI y SO es:",ECM2)
print("El mejor modelo matematico es RBI con",'TB' if ECM<ECM2 else 'SO')


La regresion lineal multiple realizada con RBI y SO resulto:
HR = -8.838190919805308 + 0.3852430183529923*RBI + -1.437846874430908e-05*SO
El error cuadratico medio de RBI y TB es: 857.5176141236892
El error cuadratico medio de RBI y SO es: 948.8365092171974
El mejor modelo matematico es RBI con TB
